# AI 사례실습과 함께하는 '드래곤볼'을 찾아서, 심화문제!
* 드래곤볼 7개를 모으면 3가지 선물이 주어진다고 합니다.
* AI 모델링 A부터 Z까지의 각 과정에 드래곤볼이 숨겨져있다고 하는데요.
* 우리는 이제 일곱 번째 마을 '심화문제' 마을에 도착했습니다.
* 그럼, 지금부터 일곱 번째 드래곤볼을 찾아볼까요?

### [상황] 일곱 번째 드래곤볼을 찾아 보겠습니다. 
* 실습문제를 해결할 때마다 단서가 주어지는데요. 
* 이 단서를 기억해두었다가 마지막에 드래곤볼을 찾을 때 활용하시면 되요.

---

In [ ]:
# 필요 라이브러리부터 설치할께요.
!pip install pandas requests seaborn xgboost gspread oauth2client

In [ ]:
# ID를 입력해주세요.
# ID는 aivle 이메일의 아이디 a로 시작하는 7글자(예를 들어 a011001)
ID = ""

---

# (실습준비) 데이터 불러오기부터


In [ ]:
import pandas as pd

# 학습/평가 데이터 로딩
df = pd.read_csv('raw_data/onenavi_train.csv',sep="|")
df_eval = pd.read_csv('raw_data/onenavi_evaluation.csv',sep="|")

# 데이터 합치기
df_total = pd.concat([df,df_eval])

# 학습/평가 데이터 로딩
df_feature = pd.read_csv("add_data/onenavi_train_feature.csv",sep="|")
df_target = pd.read_csv("add_data/onenavi_train_target.csv",sep="|")

# 학습/평가 데이터 로딩
df_evaluation_target = pd.read_csv("add_data/onenavi_evaluation_et.csv",sep="|")
df_evaluation_feature = pd.read_csv("add_data/onenavi_evaluation_feature.csv",sep="|")

In [ ]:
### Train/Test Data Split
from sklearn.model_selection import train_test_split

# train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_feature, df_target, test_size=0.20, random_state=42)

#### [실습문제1] 심화문제
* 랜덤포레스트를 활용해서 모델최적화를 하고 모델을 저장해주세요.
* 최적화된 모델은 6_model.pkl로 저장해주세요.
* 저장위치는 model 폴더입니다.
* 단서 : 비밀입니다. 모델만 저장하시면 자동으로 체크 예정이에요.


In [ ]:
# 아래에 실습코드를 작성하세요.
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *

In [ ]:
rf_gs_params = {'n_estimators': range(200, 410, 50), 'max_features': range(20, 101, 20)} # n_feature: 108

rf_gs = GridSearchCV(RandomForestRegressor(), rf_gs_params, cv=5, verbose=10)
rf_gs.fit(train_x, train_y.values.ravel())

In [ ]:
rf_gs.best_score_

In [ ]:
rf_gs.best_params_

In [ ]:
import joblib

joblib.dump(rf_gs, './model/6_model.pkl')



단서1은 따로 적지 않으셔도 됩니다. '6_model.pkl'로 저장만 하시면 자동으로 체크합니다.


#### [실습문제2] 심화문제
* Tensorflow를 활용해서 모델최적화를 하고 모델을 저장해주세요.
* 최적화된 모델은 DeeplearningModel_2.h5로 저장해주세요.
* 저장위치는 model 폴더입니다.
* 단서 : 비밀입니다. 모델만 저장하시면 자동으로 체크 예정이에요.


In [ ]:
# https://www.tensorflow.org/tutorials/keras/keras_tuner
!pip install -q -U keras-tuner==1.0.1

In [ ]:
# 아래에 실습코드를 작성하세요.
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
import IPython

In [ ]:
# input layer 제외 마음대로 변경 가능
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(108,)))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(1))
    
    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = keras.losses.MeanSquaredError(),
                  metrics = ['mae', 'mse'])
    
    return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss', 
                     max_epochs = 10,
                     factor = 3)

In [ ]:
# 학습 내용 출력값을 지워주는 callback 함수
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(train_x, train_y, epochs = 10, validation_split = .2, callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.fit(train_x, train_y, epochs = 10, validation_split=.2)

In [ ]:
pred = model.predict(test_x)

print(r2_score(test_y, pred))

In [ ]:
model.save('./model/DeeplearningModel_2.h5')



단서2는 따로 적지 않으셔도 됩니다. 'DeeplearningModel_2.h5'로 저장만 하시면 자동으로 체크합니다.


---

### [드래곤볼을 찾아라] 주어진 단서를 모아서 드래곤볼을 찾아보겠습니다.
* 소스코드 실행 후 드래곤볼 이미지 창이 열리고 "드래곤볼을 획득했습니다."라는 문구가 출력된다면 여러분은 드래곤볼을 찾은 거랍니다.


In [ ]:
# 단서를 모두 모았다면 드래곤볼을 찾아봅시다! 아래 소스코드를 실행해 주세요.
import module

module.seventhModule(ID)

In [ ]:
# 확인
from IPython.display import Image
Image("./dragonball/7성구.png", width=300, height=300)